In [ ]:
# import requirments 
import json
import pandas as pd 
import matplotlib as plt 
import seaborn as sns
import selenium
from scipy import stats
import numpy as np

import plotly.express as px

import plotly.graph_objects as go

import requests
from selenium import webdriver

import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
# Using Selenium to Scrape opensea for NFT transaction and Sales Data 

driver = webdriver.Chrome("C:\\Users\\brend\\chromedriver\\chromedriver_win32\\chromedriver.exe")
driver.get('https://opensea.io/rankings?sortBy=total_volume')

url_list = set()
n = len(url_list)
i = 0
while True:
    driver.execute_script(f"window.scrollTo(0, {1000*i});") # Scrolls to bottom of page to load new data on opensea.
    time.sleep(1)
    lines = driver.find_elements(By.CLASS_NAME, value = 'styles__StyledLink-sc-l6elh8-0') # finding collection names for API call 
    url_lst = [a.get_attribute('href') for a in lines]
    url_list = url_list.union(set(url_lst)) # appending to set for distinct collection names 
    if len(url_list) == n:
        break
    n = len(url_list)
    i += 1

driver.close()

 # cleaning collection names
url_list = list(url_list)
split = [i.split("/")for i in url_list]  
collection_names = [i[-1] for i in split]


collection_names.remove('')



collection_names.remove('account')
collection_names[:70]


   
# Collecting Data from opensea API 

NFT_dict ={}
for b in collection_names:
    url = f"https://api.opensea.io/api/v1/collection/{b}/stats"
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers)
    nft_dict = response.json()
    NFT_dict[f'{b}'] = (nft_dict['stats'])
    
stats_df= pd.DataFrame(NFT_dict).T


In [ ]:
# average price plot 
price_plot = stats_df[['average_price','floor_price','total_volume']].sort_values(by = 'average_price',ascending=False)
price_plot.to_csv('price_plot.csv')

In [ ]:
# scatter describing the correlation between amount in collection and total volume traded 
count_group = stats_df[['count', 'total_volume']].groupby('count').mean()
count_group = count_group[count_group.index <100000]
count_group = count_group[count_group.total_volume < 100000]
px.scatter(count_group, x= count_group.index, y ='total_volume', color = count_group.index)


In [ ]:
## total volume traded - 12 billion
sum_total_volume = float(stats_df.total_volume.sum())
sum_total_volume * 2538.30
## total volume traded 12 billion 

In [ ]:
# Top 10 Collections with greatest thirty day change in price  
top30change = stats_df[['thirty_day_change']].sort_values(by= 'thirty_day_change',ascending =False).head(10)

In [ ]:
### ratio of one day change vs total change 
print(stats_df.thirty_day_change.max() / stats_df.thirty_day_change.sum())
stats_df[stats_df['thirty_day_change']==stats_df['thirty_day_change'].max()]
df1 = stats_df[['thirty_day_change']]

df1['30_day_ratio'] = df1[['thirty_day_change']]/df1['thirty_day_change'].sum()
df1=df1.sort_values(by='30_day_ratio',ascending = False).head(10)
px.pie(df1,values = '30_day_ratio' , names = df1.index)
max_30_change = df1[['thirty_day_change']].sort_values(by= 'thirty_day_change',ascending=False).head(10)




In [ ]:
### highest grossing NFTs 
most_profit =stats_df.sort_values(by='total_volume',ascending =False).head(20)
px.bar(most_profit, x=most_profit.index,y='total_volume')


In [ ]:
## difference between number of total sales and the num of owners 

diff = stats_df[['num_owners','total_sales','total_volume']]

diff['num-sales'] = diff['total_sales'] - diff['num_owners']

diff = diff.drop('rarible') # dropping outlier collection. 
diff['num-sales'].median()
fig6= px.bar(diff.sort_values(by = 'total_volume',ascending = False).reset_index().tail(20), x = 'index', y= 'num-sales')
fig5= px.bar(diff.sort_values(by = 'total_volume',ascending = False).reset_index().head(20), x = 'index', y= 'num-sales')
fig5=fig5.add_hline(y=11974.0)


In [ ]:

### amount of nfts  in  the top 10 selling NFT collections   -- line is median() amount of nfts in top 100 nfts 

sup_plt = stats_df[['total_volume','total_supply']].sort_values(by= 'total_volume',ascending =False).head(10)
sup_plt = sup_plt.reset_index()
sup_plt.total_supply.median()
fig1 = px.bar(sup_plt, x="index", y= "total_supply", labels={
                     "index": "Collection Names",
                     "total_supply": "# of NFTs in collection"},title = "Total Supply of Top Selling Collections").update_xaxes(categoryorder='total descending')
fig1=fig1.add_hline(y=34661.5)





In [ ]:
# total number of NFTs in collection removing video game collection outliers 
scat=stats_df[['total_sales','total_volume','count','average_price']]
scat = scat.loc[scat['count']<60000]
scat_fig = px.box(scat,y='count',labels = {'count':'# of NFTS in collection'})
scat_fig.show()

In [ ]:
# boxplot of top 100 nft collections average price 

px.box(scat,y='average_price', labels = {'average_price':'Average Price of NFT (ETH)'})

In [ ]:
# Top 10 nfts with highest total sales 
max_sales =stats_df.total_sales.sort_values(ascending = False).to_frame().head(10)
max_sales.to_csv('max_sales.csv')


In [ ]:
# reading in Historical NFT data 
historical_nft_data = pd.read_csv("C:\\Users\\brend\\NYCDSA\\Projects\\NFT\\NFT-data-analysis-\\archive\\NFT_Sales.csv")
hist_nft = pd.DataFrame(historical_nft_data)
hist_nft.head(25)


In [ ]:
# converting data column type
type(hist_nft.iloc[0,0])
hist_nft.Date=pd.to_datetime(hist_nft.Date)

In [ ]:
# Cleaning Historical NFT data 
hist_nft.isnull().sum()
hist_nft.Active_Market_Wallets_cumsum = hist_nft.Active_Market_Wallets_cumsum.fillna(hist_nft.Active_Market_Wallets_cumsum.mean())
hist_nft.Secondary_Sales_cumsum = hist_nft.Secondary_Sales_cumsum.fillna(hist_nft.Secondary_Sales_cumsum.mean())
hist_nft.AverageUSD_cum = hist_nft.AverageUSD_cum.fillna(hist_nft.AverageUSD_cum.mean())
hist_nft.Sales_USD  = hist_nft.Sales_USD.fillna(hist_nft.Sales_USD.mean())
hist_nft.Number_of_Sales = hist_nft.Number_of_Sales.fillna(hist_nft.Number_of_Sales.mean())
hist_nft.Active_Market_Wallets = hist_nft.Active_Market_Wallets.fillna(hist_nft.Active_Market_Wallets.mean())
hist_nft.Primary_Sales = hist_nft.Primary_Sales.fillna(hist_nft.Primary_Sales.mean())
print(hist_nft.isnull().sum())

In [ ]:
# Subsetting historical nft data. 

date_line = hist_nft.set_index('Date')
date_line = date_line.loc['2020-01-01':,:]['Active_Market_Wallets_cumsum'].reset_index().dropna()


In [ ]:
# line plots of NFT collection sales over time

multi_line= hist_nft.loc['2020-01-01':,:][['Sales_USD_cumsum','AverageUSD_cum','Date','Secondary_Sales_cumsum','Primary_Sales_cumsum',"Number_of_Sales_cumsum",'Active_Market_Wallets_cumsum']].dropna()
#multi_line = multi_line.reset_index()
multi_line["Date"] = pd.to_datetime(multi_line.Date)
multi_line['Date'] = multi_line['Date'].apply(lambda x: x.strftime('%Y-%m'))

#multi_line=multi_line.set_index('Date', inplace=True)
multi_line = multi_line[['Sales_USD_cumsum','AverageUSD_cum','Date','Secondary_Sales_cumsum','Primary_Sales_cumsum',"Number_of_Sales_cumsum",'Active_Market_Wallets_cumsum']].groupby('Date').mean()
multi_line['secondary_sales_ratio'] = multi_line.Secondary_Sales_cumsum / multi_line.Number_of_Sales_cumsum
multi_line['primary_sales_ratio'] = multi_line.Primary_Sales_cumsum / multi_line.Number_of_Sales_cumsum


In [ ]:
# Creating Secondary Sales Features 
stackedline =hist_nft[['Date','Number_of_Sales','Active_Market_Wallets','Primary_Sales']]
stackedline["Date"] = pd.to_datetime(stackedline.Date)
stackedline['Date'] = stackedline['Date'].apply(lambda x: x.strftime('%Y-%m'))
stackedline = stackedline.set_index('Date')
stackedline =stackedline[['Number_of_Sales','Active_Market_Wallets','Primary_Sales']].groupby(stackedline.index).mean()
stackedline['Secondary_sales'] =stackedline['Number_of_Sales']-stackedline['Primary_Sales']
stackedline.to_csv('stackedline.csv')

stackedline

In [ ]:
# Identifying collections with greater secondary sales than primary sales. 
stackedline['Secondary_sales'] =stackedline['Number_of_Sales']-stackedline['Primary_Sales']
stackedline.loc[stackedline.Secondary_sales > stackedline.Primary_Sales]

In [ ]:
# isolatiing the NFT marketplace explosion 

nft_spike = multi_line
nft_spike= nft_spike.loc['2020-01':,:][['Sales_USD_cumsum']].dropna()
nft_spike = nft_spike.loc['2021-02':,:]


In [ ]:
### sum of active market wallets grouped by date 
wallet_hist = multi_line.reset_index()
wallet_hist["Date"] = pd.to_datetime(wallet_hist.Date)
wallet_hist['Date']= wallet_hist['Date'].apply(lambda x: x.strftime('%Y'))
zz= wallet_hist[['Active_Market_Wallets_cumsum','Date']].groupby('Date').sum()
px.histogram(zz, y='Active_Market_Wallets_cumsum',x=zz.index)

In [ ]:
bb = wallet_hist[['Secondary_Sales_cumsum','Date']].groupby('Date').sum()
bb.Secondary_Sales_cumsum=bb.Secondary_Sales_cumsum.astype(int)


px.bar(bb, x =bb.index, y = 'Secondary_Sales_cumsum' )


In [ ]:
zz['Active_Market_Wallets_cumsum']
zz.sum()

In [ ]:
# Identifying dates with largest primary sales. 
primary_by_date = wallet_hist[['Primary_Sales_cumsum','Date']].groupby('Date').sum()
primary_by_date.Secondary_Sales_cumsum=primary_by_date.Primary_Sales_cumsum.astype(int)


In [ ]:

sales_line = hist_nft.set_index('Date')
sales_line = sales_line.loc['2020-01-01':,:][['Sales_USD_cumsum',"Number_of_Sales_cumsum",'Active_Market_Wallets_cumsum']].reset_index().dropna()


In [ ]:
# line graph showing cumalative sales from 2017-2018 
fig = px.line(hist_nft,  x=hist_nft.Date, y=hist_nft.Sales_USD_cumsum, title = "Cumulative USD sales 2017-2021")
fig.show()

In [ ]:
## increase in the amount of active wallets 2017 -2021 

fig2 = px.line( x=hist_nft.Date, y=hist_nft.Active_Market_Wallets_cumsum, title = " Amount of Active Market Wallets ")
fig2.show()

In [ ]:
# Overall NFT transaction sales with info on transactions, buyers and token owners 

nft_sales = pd.read_csv(r"C:\Users\brend\NYCDSA\Projects\NFT\NFT-data-analysis-\nft_sales.csv")
sales_df = pd.DataFrame(nft_sales)

sales_df.Sales =sales_df['Sales'].str.replace(',','')
sales_df.Sales =sales_df['Sales'].str.replace('$','')
sales_df.Sales= sales_df.Sales.astype(float)
sales_df.Buyers =sales_df['Buyers'].str.replace(',','')
sales_df.Buyers =sales_df['Buyers'].str.replace('$','')
sales_df.Txns =sales_df['Txns'].str.replace(',','')
sales_df.Txns =sales_df['Txns'].str.replace('$','')
sales_df.Owners =sales_df['Owners'].str.replace(',','')
sales_df.Owners =sales_df['Owners'].str.replace('$','')
sales_df.Buyers.astype(int)
sales_df.Owners.astype(float)
sales_df.Txns.astype(int)
sales_df

sales_df = sales_df.set_index('Collections')

b = sales_df.sort_values(by='Sales',ascending=False).head(20)

b['Txns'].median()

d = sales_df.sort_values(by='Sales',ascending=True).head(20)
d.to_csv('collection_sales_buyers_least.csv')

print(b.Owners.median())

print(b.Buyers.median())




In [ ]:
### merged data frames to include count along with total txns, buyers , and owners 
sales_df.index= sales_df.index.str.lower()

coll_count = stats_df['count'].to_frame()
coll_count.median()
coll_count_sales = coll_count.merge(sales_df, left_index=True, right_index= True, how='inner')
coll_count_sales
coll_count_sales.to_csv('coll_count_sales.csv')
